In [6]:
import os, zipfile
import pandas as pd

# Download Data from [binance-public-data](https://github.com/binance/binance-public-data/tree/master/python)

Download BTCUSDT and ETHUSDT for all available history for intervals of 1m, 3m, 5m, 15m, 30m


In [1]:
!python binance-public-data/python/download-kline.py -t spot -s BTCUSDT ETHUSDT -i 1m 3m 5m 15m 30m -skip-daily 1

Found 2 symbols
[1/2] - start download monthly BTCUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-07.zip

File Download: /Users/HSY/PhD/backtrader-crypto-rl/binance-public-data/python/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2017-08.zip
[##################################################]
File Download: /Us

# Read Downloaded Data

The downloaded data can be found in `binance-public-data/data/`

In [11]:
# List of symbols to merge
symbols = ['BTCUSDT', 'ETHUSDT']

# Frequencies
freqs = ['1m', '3m', '5m', '15m', '30m']

# List to store individual DataFrames
rawdfs = []

# Loop through each freq
for freq in freqs:
    # Loop through each symbol
    for symbol in symbols:
        directory = f'binance-public-data/python/data/spot/monthly/klines/{symbol}/{freq}/'
        
        # Loop through each zip file in the directory
        for file_name in os.listdir(directory):
            if file_name.endswith('.zip'):
                with zipfile.ZipFile(os.path.join(directory, file_name), 'r') as zip_ref:
                    # only one CSV file in each zip archive
                    csv_file = zip_ref.namelist()[0]
                    with zip_ref.open(csv_file) as csv_fp:
                        # Read the CSV data into a DataFrame
                        temp_df = pd.read_csv(csv_fp, header=None)
                        temp_df.columns = [
                            'open_time', 'open', 'high', 'low', 'close', 'volume', 
                            'close_time', 'quote_asset_volume', 'number_of_trades', 
                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                        ]
                        # temp_df['date'] = pd.to_datetime(temp_df['close_time'], unit='ms').dt.strftime('%Y-%m-%d')
                        temp_df = temp_df.rename(columns={"close_time": "time"})
                        temp_df['tic'] = symbol
                        temp_df['itvl'] = freq
                        rawdfs.append(temp_df[['time', 'open', 'high', 'low', 'close', 'volume', 'tic', 'itvl']])

# Concatenate all DataFrames into a single DataFrame
rawdf = pd.concat(rawdfs, ignore_index=True)

# Count the number of unique 'tic' values per date
tic_counts = rawdf.groupby('time')['tic'].nunique()

# Filter the DataFrame to keep only rows where all 'tic' values participate
df = rawdf[rawdf['time'].isin(tic_counts[tic_counts == len(rawdf['tic'].unique())].index)]
# Only wanted columns
df = df[['time', 'open', 'high', 'low', 'close', 'volume', 'tic', 'itvl']]
df['datetime'] = pd.to_datetime(df['time'], unit='ms')

df = df.sort_values(['time', 'tic', 'itvl'],ignore_index=True)

# df = df.head(100000)
df

,time,open,high,low,close,volume,tic,itvl,datetime
0,1502942459999,4261.48,4261.48,4261.48,4261.48,1.775183,BTCUSDT,1m,2017-08-17 04:00:59.999
1,1502942459999,301.13,301.13,301.13,301.13,0.426430,ETHUSDT,1m,2017-08-17 04:00:59.999
2,1502942519999,4261.48,4261.48,4261.48,4261.48,0.000000,BTCUSDT,1m,2017-08-17 04:01:59.999
3,1502942519999,301.13,301.13,301.13,301.13,2.757870,ETHUSDT,1m,2017-08-17 04:01:59.999
4,1502942579999,4280.56,4280.56,4280.56,4280.56,0.261074,BTCUSDT,1m,2017-08-17 04:02:59.999
...,...,...,...,...,...,...,...,...,...
10303403,1693526399999,1646.73,1646.96,1645.08,1645.76,1211.401200,ETHUSDT,15m,2023-08-31 23:59:59.999
10303404,1693526399999,1645.76,1645.77,1645.76,1645.76,107.315300,ETHUSDT,1m,2023-08-31 23:59:59.999
10303405,1693526399999,1644.32,1646.96,1644.04,1645.76,2369.983200,ETHUSDT,30m,2023-08-31 23:59:59.999
10303406,1693526399999,1646.26,1646.26,1645.76,1645.76,203.557700,ETHUSDT,3m,2023-08-31 23:59:59.999
